## Setting up
### Libraries


In [ ]:
# 설치되지 않은 라이브러리의 경우, 주석 해제 후 코드를 실행하여 설치
# !pip install segmentation-models-pytorch
# !pip install albumentations

In [ ]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import segmentation_models_pytorch as smp
import argparse
import sys
import os
import torch.optim as optim
from torch.utils.data import DataLoader
import albumentations as albu
import json
import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

from datetime import datetime, timezone, timedelta
import random

from torch.utils.data import Dataset
import json
import cv2
from PIL import Image, ImageDraw
from glob import glob

### etc..
- working directory  
  |--code.ipynb  
  |--data/  
  |--|--train/  
  |--|--|--images/  
  |--|--|--|--...  
  |--|--|--labels.json  
  |--|--test/  


#### fill the blanks!


In [ ]:
# seed
RANDOM_SEED = ##### value #####
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Set device
os.environ["CUDA_VISIBLE_DEVICES"] = ##### value #####
# torch.cuda.set_device(1)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# directory
DATA_DIR = ##### directory #####

## Dataload

you can add Augmentation, preprocessing part


In [ ]:
# hyper-parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 32
VAL_RATIO = 0.1

#### fill the blanks!
  (`##### code #####` part)

In [ ]:
def polygon_to_mask(json_file, filename):
   
        # n : n.jpg
        n = int(filename.split('.')[0])
        file_dict = json_file['annotations'][n]
           
        class_name = []
        polygon_dict = []
        
        polygon_dict = []
        class_name = []
        mask = []
        
        if len(file_dict['objects']) == 0:
            mask_ = []
            img = Image.new('L', (720,480), 'black')
            img = img.resize((512,512))
            mask_.append(np.array(img))
            class_name.append(0)
            mask_[0] = np.where(mask_[0]!=0, 0, mask_[0])
            
        else:
            for i in range(len(file_dict['objects'])):
                mask_ = []
                clss = file_dict['objects'][i]['class']
                poly = np.array(file_dict['objects'][i]['polygon'], np.float32)
                                
                img = Image.new('L', (720,480), 'black')
                
                ImageDraw.Draw(img).polygon(poly, outline='white', fill='white')
                img = img.resize((512,512))
                mask_.append(np.array(img))
                idx = len(mask_)-1
                mask_[idx] = np.where(mask_[idx]!=0, clss, mask_[idx])             
                class_name.append(clss)
                
            if np.any(class_name[:]) > 4:
                print("filename : ", filename, " class : ", clss_name)                 
        
        # combine masks
        mask = ##### code #####

        return mask, class_name

In [ ]:
from torch.utils.data import Dataset
from glob import glob
from sklearn.model_selection import train_test_split

class CustomDataset(Dataset):
    def __init__(
            self, 
            mode = 'train',
            data_dir = DATA_DIR,
            val_ratio = VAL_RATIO,
            augmentation=None, 
            preprocessing=None
            
    ):
        # 데이터 위치 설정
        path = ##### 경로 #####
        self.path = path
        self.json_file = json.load(open(os.path.join(path,'labels.json')))
        df = self.get_filelist()
        
        # file_names : 파일명 리스트, 이미지-레이블-마스크 모두 파일명으로 매칭됨
            
        train_df, val_df = train_test_split(df,test_size=val_ratio, shuffle=True, random_state=42)       
        
        # TRAIN_RATIO 로 학습/추론 데이터 분할
        if mode == 'train':
            self.file_names = train_df
            
        elif mode =='valid':
            self.file_names = val_df

        self.augmentation = augmentation
        self.preprocessing = preprocessing
   
    def get_filelist(self):
        
        filenames = os.listdir(os.path.join(self.path, 'train'))
        filenames = [file for file in filenames if ".jpg" in file]
        
        return filenames
        

    # i번째 이미지와 마스크를 리턴
    def __getitem__(self, i):
        
        # read data
        img_fps = os.path.join(self.path, 'train', self.file_names[i])
        image = ##### 코드 #####
        image = cv2.resize(image, (512,512))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask = []
        
        mask, c = polygon_to_mask(self.json_file, self.file_names[i])

        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask.squeeze(0)
    
    # 이미지 파일 수를 리턴
    def __len__(self):
        return len(self.file_names)

In [ ]:
# 데이터 로드
def img_to_tensor(x, **kwargs):
    return x.transpose(2, 1, 0).astype('float32')

def mask_to_tensor(x, **kwargs):
    return x.transpose(0 ,2, 1).astype('int32')

# smp의 전처리 함수를 데이터 로드에 사용하기 위한 변환
def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=img_to_tensor, mask=mask_to_tensor),
    ]
    return albu.Compose(_transform)

# Building the model


In [ ]:
# parameters
EPOCHS = ##### value #####
LEARNING_RATE = ##### value #####
WEIGHT_DECAY = ##### value #####
NUM_WORKERS = ##### value #####
PIN_MEMORY = True

ENCODER = 'efficientnet-b0' # 예시
ENCODER_WEIGHTS = 'imagenet' # 예시
CLASSES = ['background', 'black', 'gray', 'white', 'fire']
ACTIVATION = None # 예시

In [ ]:
model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [ ]:
# Load dataset & dataloader using preprocessing fn

train_dataset = CustomDataset(
    mode = 'train',
    data_dir = DATA_DIR,
    preprocessing=get_preprocessing(preprocessing_fn),
)

val_dataset = CustomDataset(
    mode = 'valid',
    data_dir = DATA_DIR,
    preprocessing=get_preprocessing(preprocessing_fn),
)

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=EVAL_BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=False)

In [ ]:
print(len(train_dataset))
print(len(val_dataset))

In [ ]:
import time
import torch.nn.functional as F

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def mIoU(pred_mask, mask, smooth=1e-10, n_classes=5):
    with torch.no_grad():
        pred_mask = F.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes): #loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            if true_label.long().sum().item() == 0: #no exist label in this loop
                iou_per_class.append(np.nan)
            else:
                intersect = torch.logical_and(true_class, true_label).sum().float().item()
                union = torch.logical_or(true_class, true_label).sum().float().item()

                iou = (intersect + smooth) / (union +smooth)
                iou_per_class.append(iou)
        return np.nanmean(iou_per_class)

def fit(epochs, model, train_loader, val_loader, criterion, optimizer, scheduler, patch=False):
    torch.cuda.empty_cache()
    train_losses = []
    test_losses = []
    val_iou = []; 
    train_iou = [];
    lrs = []
    min_loss = np.inf
    decrease = 1 ; not_improve=0

    model.to(DEVICE)
    fit_time = time.time()
    for e in range(epochs):
        since = time.time()
        running_loss = 0
        iou_score = 0
        #training loop
        model.train()
        for i, data in enumerate(tqdm(train_loader)):
            #training phase
            image_tiles, mask_tiles = data
            mask_tiles = mask_tiles.long().squeeze(1)
            
            if patch:
                print(image_tiles.size())
                bs, c, h, w = image_tiles.size()

                image_tiles = image_tiles.view(-1,c, h, w)
                mask_tiles = mask_tiles.view(-1, h, w)
            
            image = image_tiles.to(DEVICE); mask = mask_tiles.to(DEVICE);
            #forward
            output = ##### 코드 #####
            loss = ##### 코드 #####
            #evaluation metrics
            iou_score += mIoU(output, mask)
            #backward
            loss.backward()
            optimizer.step() #update weight          
            optimizer.zero_grad() #reset gradient
            
            #step the learning rate
            lrs.append(get_lr(optimizer))
            scheduler.step() 
            
            running_loss += loss.item()
            
        else:
            model.eval()
            test_loss = 0
            val_iou_score = 0
            #validation loop
            with torch.no_grad():
                for i, data in enumerate(tqdm(val_loader)):
                    #reshape to 9 patches from single image, delete batch size
                    image_tiles, mask_tiles = data
                    mask_tiles = mask_tiles.long()

                    if patch:
                        print(image_tiles.size())
                        bs, c, h, w = image_tiles.size()

                        image_tiles = image_tiles.view(-1,c, h, w)
                        mask_tiles = mask_tiles.view(-1, h, w)
                    
                    image = image_tiles.to(DEVICE); mask = mask_tiles.to(DEVICE);
                    output = ##### 코드 #####
                    #evaluation metrics
                    val_iou_score +=  mIoU(output, mask)
                    #loss
                    loss = ##### 코드 #####                                 
                    test_loss += loss.item()
            
            #calculatio mean for each batch
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(val_loader))


            if min_loss > (test_loss/len(val_loader)):
                print('Loss Decreasing.. {:.3f} >> {:.3f} '.format(min_loss, (test_loss/len(val_loader))))
                min_loss = (test_loss/len(val_loader))
                decrease += 1
                if decrease % 5 == 0:
                    print('saving model...')
                    torch.save(model, 'model_best.pt'.format(val_iou_score/len(val_loader)))
                    

            if (test_loss/len(val_loader)) > min_loss:
                not_improve += 1
                min_loss = (test_loss/len(val_loader))
                print(f'Loss Not Decrease for {not_improve} time')
                if not_improve == 7:
                    print('Loss not decrease for 7 times, Stop Training')
                    break
            
            #iou
            val_iou.append(val_iou_score/len(val_loader))
            train_iou.append(iou_score/len(train_loader))
            print("Epoch:{}/{}..".format(e+1, epochs),
                  "Train Loss: {:.3f}..".format(running_loss/len(train_loader)),
                  "Val Loss: {:.3f}..".format(test_loss/len(val_loader)),
                  "Train mIoU:{:.3f}..".format(iou_score/len(train_loader)),
                  "Val mIoU: {:.3f}..".format(val_iou_score/len(val_loader)),
                  "Time: {:.2f}m".format((time.time()-since)/60))
        
    history = {'train_loss' : train_losses, 'val_loss': test_losses,
               'train_miou' :train_iou, 'val_miou':val_iou,
               'lrs': lrs}
    print('Total time: {:.2f} m' .format((time.time()- fit_time)/60))
    return history


In [ ]:
criterion = nn.CrossEntropyLoss() #예시
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY) #예시
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, LEARNING_RATE, epochs=EPOCHS,
                                            steps_per_epoch=len(train_loader)) #예시

# train start
history = fit(EPOCHS, model, train_loader, valid_loader, criterion, optimizer, scheduler)

In [ ]:
torch.save(model, 'model_final.pt')

## Predict



In [ ]:
# sample_submission
sub_json = json.load(open('sample_submission.json'))

In [ ]:
sub_json['annotations'][:5]

In [ ]:
TEST_DATA_DIR = ##### 경로 #####

In [ ]:
class TestDataset(Dataset):
    def __init__(
            self, 
            mode = 'test',
            data_dir = TEST_DATA_DIR,
            classes=None,
            augmentation=None, 
            preprocessing=None
            
    ):
        # 데이터 위치 설정
        path = ##### 경로 #####
        
        # file_names : 파일명 리스트, 이미지-레이블-마스크 모두 파일명으로 매칭됨
        self.file_names = glob(os.path.join(path, '*.jpg'))
        
        self.images_fps = [os.path.join(path, 'images', i) for i in self.file_names]

        self.augmentation = augmentation
        self.preprocessing = preprocessing
   

    # i번째 이미지와 마스크를 리턴
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.resize(image, (512,512))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image)
            image = sample['image']
            
        return image
    
    # 이미지 파일 수를 리턴
    def __len__(self):
        return len(self.file_names)

In [ ]:
# 데이터 로드
def img_to_tensor(x, **kwargs):
    return x.transpose(2, 1, 0).astype('float32')

def mask_to_tensor(x, **kwargs):
    return x.transpose(0 ,2, 1).astype('float32')

# smp의 전처리 함수를 데이터 로드에 사용하기 위한 변환
def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=img_to_tensor),
    ]
    return albu.Compose(_transform)

In [ ]:
test_dataset = TestDataset(
    mode = 'test',
    data_dir = TEST_DATA_DIR,
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

test_loader = DataLoader(test_dataset, batch_size=EVAL_BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=False, drop_last=False)

#### 코드 채워넣기
- 경로 설정을 완성하세요.  
  (`##### 경로 #####` 부분)

In [ ]:
model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

MODEL_DIR = ##### 경로 #####
model = torch.load(MODEL_DIR)
model.to(DEVICE)

In [ ]:
# 모델 결과 저장
preds = []


with torch.no_grad():
    model.eval()
    for i, batch_data in enumerate(tqdm(test_loader)):
        x = batch_data.to(DEVICE)
        output = model.predict(x)
        mask = torch.argmax(output, dim=1)
        preds.extend(mask)

In [ ]:
pred_result = {'annotations':[]}
wratio = 720/512
hratio = 480/512

for i, file in enumerate(test_dataset.file_names):
    
    file_name = ##### 코드 #####
    objects = []
    pred = np.array(preds[i].cpu()) 
    
    # background 제외
    for c in range(1,len(CLASSES)):
        clss = np.full((512,512),c) #필터
        layer = np.equal(pred, clss).astype('int')
        if layer.sum()!=0:
           
            # findContours에 들어갈 이미지 형태로 변환
            layer = np.array(layer*255).astype('uint8')
            contours, _ = cv2.findContours(layer, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

            # contours[2] = contour의 좌표 리스트
            polygon=[]

            for a in contours:
                for point in a:
                    x = point[0][0]*wratio
                    y = ##### 코드 ######
                    x = np.round(x, 1)
                    y = np.round(y, 1)
                    polygon.append([x,y])

            # 채점은 점이 3개 이상이어야만 하기 때문에, 폐곡선이 아닌 경우 점을 추가
            if len(polygon) == 1:
                polygon.append(##### 코드 #####)
                polygon.append(polygon[0])
            elif len(polygon) == 2:
                polygon.append(polygon[0])

            objects.append({'class':c, 'polygon': polygon})
            
    
    pred_result['annotations'].append({'file_name':file_name, 'objects':objects})
    
print('Predict Completed!')

In [ ]:
json.dump(pred_result,open(##### 경로 #####, 'w'))